# Fast.ai Lessons 9 Roundup
- categories: [Fast.ai2]

# Lesson Overview
(this lesson is in progress)
Runthrough stable diffusion, then detail behind how it works. There will be some handwaving for a few lessons, because it will take multiple lessons to learn it all from scratch. The point is, we want to build intuition first and see the full picture then narrow into the details as needed.
Jeremy said he expects people to spend on average, 10 hours a lesson.
- This lesson is 3 lessons. This general video, a deeper code dive called lesson 1A, and another for the maths.

# The topics covered, briefly


# Lecture Notes
- GPU needs have increased for this course. Google collab pricing has increased. Maybe use my laptop GPU or the uni one. The four options: Colab, Paperspace Gradient, Lambda Labs, Jarvis Labs. Lambda GPU offer:$1.10/hour. Lambda is cheaper than everything else but it nor collab allow you to pause. This can change again too. Funnily enough, collab is skyrocking in price because everyone is using it for stable diffusion. 
- Training GPUs need 16/24 gb. My laptop is only 12gb, and it's probably not simple to optimise it for ML. I might be able to do everything that isn't training on my PC.
- Probably should join/create a study group to enjoy the course more.
- This course covers details you need to know to be a researcher or custom setup for industry. 
- Stable diffusion is moving so quickly, the details change. Literally in the last 24hrs, 2 important new papers came out. The number of steps has gone down from from 250 from 1000! And something else changed to make it 10x-20x faster! I wonder if this is a result directly from SD's open source nature. The research community may well be able to improve SD much faster then say OpenAi can improve DALL-E.
- But after this first lesson, we go into the foundations which won't change, so the research update details won't be hard to understand.
- We can fine-tune SD using dreambooth, to put any object or person into an image! strmr.com is a service to do so!
- Diffusion-nbs notebook and repo. It contains many things to play with and then understand what's possible. It's worth testing it. There's notebooks with tons of parameters we don't get understand yet. I think this is the philosophy that, to understand a black box we should first get intuition for it's inputs and outputs, then kneed into the details. 
- It's not actually easy to know what prompts to give SD. The best way to learn is to look at other people's prompts and outputs lexica.art gives plenty examples.
- First start by cloning Stable Diffusion notebook.
- Diffusers is the HuggingFace module for this. HuggingFace has had really good packages for a while now.
- We use the SD pipeline. A pipeline is similar to a fast.ai learner. It does many things automatically. We can save pipelines like we save learners. We can also save pipelines and upload them into HuggingFace's cloud server. Like this, we can browse and download pipeline's from HuggingFace, and also upload our own and share them. 
- If we use paperspace or lambda, the pipeline we downloaded will be saved, but if we use collab then no.
- If we use the same pipeline with the same random seed, we get the same result. 
- Generally, diffusion works as follows: we start with random noise, and each step we get slightly less noisy and towards the result we want, through many steps, e.g. 51. Although after yesterday it's now 3-4...
- Why don't we just do it in one step? Our models aren't smart enough, but considering it's not 3-4 steps from 1000, maybe they will be.
- The exact same prompt, four times, we pass them into the pipeline. The pipeline has a variable, guidance_scale: 'to what degree should we focus on the caption versus just creating the image'.
- Too low guidance won't make images of the prompt, too high could be strange depending on the implementation of SD.
- It works by making an image without g, then image with g, and find the average of it. 
- Negative prompts are similar. Make two images, and subtract one from another. Say we want to remove blue from the image. Subtract a prompt blue from it.
- Passing images into SD instead of prompts: We use image to image pipelines. Instead of starting diffusion with random noise from scratch, it starts with a noisy version of the input image, using it as a guiding point. It uses a parameter strength, which is similar to g I think, how strongly to follow the image.
- New technique: Let's take an image and pass it into SD, then pass the resulting image into SD again! We can use this to change styles of images for example!
- Fine-tuning: We take the models in the pipeline and pass it our own images and captions. Lambdalabs have a blog post guide for it.
- Specical kind of fine-tuning: textual inversion. We find tune a single embedding. Create a new embedding for some concept images we want. E.g. 'watercolour', and add that token to the text model, and train embeddings for it using our images. Then we can have a prompt "Woman in the style of <watercolour>".
- Dreambooth is similar to textual inversion. It finds a existing token in the embeddings that is barely used, instead of making a new one, and fine tunes it. 
- Not sure why/if Textual inversion or Dreambooth is better.


# Questions


# Links
- As I am doing this lesson as it is released privately in real time, I cannot share the lesson link.